# Objective

Gemerate prime numbers in-between 2 and the number N inclusive. Test multiple approaches and verify the performances.

## Findings

For N=10000


1. Set minus/diff operation is fast. List remove is slow.
   - Set vesion: ```4.13 ms ± 828 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)```
   - List version: ```523 ms ± 55.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)```
   - NumPy version: ```219 ms ± 44.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)```
   <br><br>
2. numpy.meshgrid() requires larger memory (> 6x) which can crash Python if N is larger.
   - Set version: ```peak memory: 94.82 MiB, increment: 0.22 MiB```
   - Set version: ```peak memory: 94.98 MiB, increment: 0.02 MiB```
   - Numpy version: ```peak memory: 690.38 MiB, increment: 572.00 MiB```


In [1]:
from typing import (
    List,
    Iterable,
    Set
)
import sys
import random
import logging
import numpy as np
import matplotlib.pyplot as plt 

# Constant

In [3]:
# Max integer N up to which (inclusive) to generate the primes 
N = 10000
LOG_LEVEL = logging.ERROR

# Setup

In [2]:
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(linewidth=80) 
 
#!conda install line_profile -c conda-forge
%load_ext line_profiler
%load_ext memory_profiler

In [4]:
logging.basicConfig(level=LOG_LEVEL)
LOGGER = logging.getLogger(__name__)

# Approach 01

* $\mathbb{Z}$: Set of all integer numbers 
* $\mathbb{P}$: Set of prime numbers
* $\mathbb{D}$: Set of divisible numbers

For each prime number ```p``` (start from 2) in $\mathbb{P}$, remove its divisible numbers $\mathbb{D_p}$ (2*p, 3*p, ...) from $\mathbb{Z}$. 

$\mathbb{Z_p} = \mathbb{Z} - \bigcup_{p=2}^{\infty} D_{p}$

After removing the union of the divisible numbers $\bigcup_{p=2}^{p} D_{p}$ of prime numbers up to ***p***, the minimum number in $Z_p$ is the next prime number next to ```p```.

<img src="images/integer_is_prime_plus_divisibles.JPG" align="left" width=650/>

## SET version 
Using Set instead of List to manage the numbers and primes.

In [5]:
def primes(n: int) -> list:
    numbers = set(range(2, n+1))
    primes = set([])
    while len(numbers) > 0:
        LOGGER.debug("len(numbers) is %s", len(numbers))

        # --------------------------------------------------------------------------------
        # The minimum number in the set is always the smallest prime number.
        # --------------------------------------------------------------------------------
        prime: int = min(numbers)
        primes.add(prime)
        LOGGER.debug("prime [%s] for the current numbers %s", prime, numbers)

        # ================================================================================
        # Remove all the numbers divisible by the prime.
        # ================================================================================
        numbers.remove(prime)

        # --------------------------------------------------------------------------------
        # Suppose 'numbers' is a List, instead of Set:
        # When prime=p, the numbers which are still in 'numbers' and between (p, p^2), that 
        # is numbers[p: p^2 +1], are all primes.
        #
        # Becuse for p, divisible numbers are p * P(C(primes, k)), where P(C(primes,k)) is
        # multiplications of all the possible combinations of primes C(primes, k) k:1,2,..
        # e.g. for primes[2,3,5] and k:1,2,3:
        # C([2,3,5], 1) = (2,3,5)        
        # C([2,3,5], 2) = C([2,3,5], 1) + (  2^2,   2*3,   2*5,   3^2,   3*5,   5^2) 
        # C([2,3,5], 3) = C([2,3,5], 2) + (2*2^2, 2^2*3, 2^2*5, 2*3^2, 2*3*5, 2*5^2, 3*3^2, 3*3*5, 3*5^2, 5^3)
        # 
        # For p=5, all divisible numbers less than p^2, those within range(2, 25), are
        # P(C([2,3,5], k=1,2,3)) and the primes in ( numbers - P(C([2,3,5], k=1,2,3)) )
        # P([2], k) = (2,   4,   6,  8,  10,   12,   14,   16,   18,   20,   22,   24   ) 
        # P([3], k) = (  3,      x     9,      x        15,      x        21,      x    ) 
        # P([5], k) = (       5,         x              x              x              25)
        # numbers   = (            7,       11,   13,         17,   19,         23,     )
        # 
        # Therefore, for p(e.g=5), there is no divisible number in numbers[2: p^2].
        # --------------------------------------------------------------------------------
        start = np.square(prime)
        if start > n:
            # --------------------------------------------------------------------------------
            # All renaming numbers are primes because range(p, p^2) has primes only.
            # Hence no more processing to remove divisibles are required.
            # --------------------------------------------------------------------------------
            primes.update(numbers)
            break
        else:
            divisibles = set(range(start, n+1, prime))
            LOGGER.debug("primes(): divisibles are %s", divisibles)
            
            # set(this) -= set(*others) will not cause an error when others has an element
            # which this set does not has.
            numbers -= divisibles
            LOGGER.debug("primes(): numbers after removing divisibles are %s", numbers)
    
    # return primes
    _primes = list(primes)
    _primes.sort()
    return _primes

In [6]:
%lprun \
    -T primes.log \
    -f primes \
    primes(N)

print(open('primes.log', 'r').read())


*** Profile printout saved to text file 'primes.log'. 
Timer unit: 1e-06 s

Total time: 0.009456 s
File: <ipython-input-5-8cbdd747160d>
Function: primes at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def primes(n: int) -> list:
     2         1       1093.0   1093.0     11.6      numbers = set(range(2, n+1))
     3         1         10.0     10.0      0.1      primes = set([])
     4        26         47.0      1.8      0.5      while len(numbers) > 0:
     5        26        155.0      6.0      1.6          LOGGER.debug("len(numbers) is %s", len(numbers))
     6                                           
     7                                                   # --------------------------------------------------------------------------------
     8                                                   # The minimum number in the set is always the smallest prime number.
     9                                      

In [7]:
%%timeit
primes(N)

4.13 ms ± 828 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%memit
primes(N)

peak memory: 94.82 MiB, increment: 0.22 MiB


# List version
Use List instead of Set to manage numbers and primes.

In [9]:
def primes_list_version(n: int) -> set:
    numbers = list(range(2, n+1))
    primes = []
    while len(numbers) > 0:
        LOGGER.debug("len(numbers) is %s", len(numbers))

        # The minimum number in the set is always the smallest prime number.
        prime: int = numbers[0]
        primes.append(prime)
        LOGGER.debug("prime [%s] for the current numbers %s", prime, numbers)

        # Remove all the numbers divisible by the prime.
        numbers.remove(prime)
        start = np.square(prime)
        if start > n:
            LOGGER.debug("primes(): start %s > %s. Break the loop", start, n)
            primes.extend(numbers)
            break
        else:
            divisibles = range(start, n+1, prime)
            LOGGER.debug("primes(): divisibles are %s", list(divisibles))

            for d in divisibles:
                try:
                    numbers.remove(d)
                except ValueError as e:
                    LOGGER.debug("Removing %s caused %s", d, e)
                    pass
                
            LOGGER.debug("primes(): numbers after removing divisibles are %s", numbers)
        
    return primes

In [10]:
%lprun \
    -T primes_list_version.log \
    -f primes_list_version \
    primes_list_version(N)

print(open('primes_list_version.log', 'r').read())


*** Profile printout saved to text file 'primes_list_version.log'. 
Timer unit: 1e-06 s

Total time: 0.612243 s
File: <ipython-input-9-4e380913de3f>
Function: primes_list_version at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def primes_list_version(n: int) -> set:
     2         1        300.0    300.0      0.0      numbers = list(range(2, n+1))
     3         1          2.0      2.0      0.0      primes = []
     4        26         58.0      2.2      0.0      while len(numbers) > 0:
     5        26         85.0      3.3      0.0          LOGGER.debug("len(numbers) is %s", len(numbers))
     6                                           
     7                                                   # The minimum number in the set is always the smallest prime number.
     8        26         33.0      1.3      0.0          prime: int = numbers[0]
     9        26         38.0      1.5      0.0          primes.appen

In [11]:
%%timeit
primes_list_version(N)

523 ms ± 55.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%memit
primes_list_version(N)

peak memory: 94.98 MiB, increment: 0.02 MiB


# Approach 02
## Numpy meshgrid product

The divisible numbers D are products x*y of the (x, y) grid. Then P = set(range(2, n+1)) - D.
<img src="images/divisibles_as_grid_product.JPG" align="left" width=650/>

In [4]:
from typing import List
import numpy as np


def prime_numpy_version(n: int) -> List[int]:
    """Generate prime numbers
    P = Z - D
    P: Primes
    Z: All integer numbers
    D: Divisible (non-primes)

    Args:
        n: upper bound to generate primes in-between 0 and n (inclusive)
    Returns:
        list of prime numbers
    """
    Diff = np.setdiff1d
    arm = range(2, np.floor(n / 2).astype(int) + 1)
    x, y = np.meshgrid(*([arm] * 2))

    Z = range(2, n + 1)
    D = x * y
    P = Diff(Z, D[D <= n].ravel())

    return P.tolist()

In [5]:
print(prime_numpy_version(-1))
print(prime_numpy_version(0))
print(prime_numpy_version(1))
print(prime_numpy_version(2))
print(prime_numpy_version(3))
print(prime_numpy_version(100))

[]
[]
[]
[2]
[2, 3]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [14]:
%lprun \
    -T prime_numpy_version.log \
    -f prime_numpy_version \
    prime_numpy_version(N)

print(open('prime_numpy_version.log', 'r').read())


*** Profile printout saved to text file 'prime_numpy_version.log'. 
Timer unit: 1e-06 s

Total time: 0.238017 s
File: <ipython-input-13-f75a5612e6c8>
Function: prime_numpy_version at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def prime_numpy_version(n: int) -> List[int]:
     2                                               """Generate prime numbers in-between 2 and n
     3                                               Args: 
     4                                                   n: upper bound to generate primes in-beteen 0 and n (inclusive)
     5                                               Returns: 
     6                                                   list of prime numbers
     7                                               """
     8         1         83.0     83.0      0.0      arm = range(2, np.floor(n / 2).astype(int) + 1)
     9         1     142837.0 142837.0     60.0      x, y = np.meshgrid

In [16]:
%%timeit
prime_numpy_version(N)

219 ms ± 44.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%memit
prime_numpy_version(N)

peak memory: 690.38 MiB, increment: 572.00 MiB


# Validation

Make sure all approaches produces the same.

In [15]:
benchmark = primes(N)
assert benchmark == primes_list_version(N)
assert benchmark == prime_numpy_version(N)

In [18]:
print(primes(100))

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


# Approach 03
## Numpy prime number bitmap

For $S = np.sum(D[y]),\; axis=0)$, if $S[x] == 1$ is True, x is prime. 

<img src="images/prime_bit_matrix.JPG" align="left" width=650/>
